# Convolutional Neural Networks

| Date | User | Change Type | Remarks |  
| ---- | ---- | ----------- | ------- |
| 19/02/2025  | Martin | Created   | Continuing chapters for CNN | 

# Content

* [Applying StyleNet and Neural Style Project](#applying-stylenet-and-neural-style-project)

# Applying StyleNet and Neural Style Project

## StyleNet

* A procedure that attempts to learn an image style from one picture and apply it to a second picture, while keeping the second image structure in tact
* Find intermediate CNN nodes that correlate strongly with the "style" and separate it from the contents of the image
* Based on a 2015 paper by Leon Gatys
* Train the style layers on the first picture and contents on the second

In [2]:
import imageio
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt
import matplotlib as mpl
import IPython.display as display
from skimage.transform import resize
from tensorflow import keras

import os
os.environ["TF_CPP_MIN_LOG_LEVEL"] = "3"
os.environ["GRPC_VERBOSITY"] = "ERROR"
os.environ["GLOG_minloglevel"] = "2"

In [ ]:
# Images files
path = './images/'
content_image_file = path + "book_cover.jpg"
style_image_file = path + "starry_night.jpg"

# Read and convert images
content_image = imageio.imread(content_image_file)
content_iamge = tf.image.convert_image_dtype(content_image, tf.float32)
style_image = imageio.imread(style_image_file)
style_image = tf.image.convert_image_dtype(style_image, tf.float32)

# Get shape of target and make the style image the same
target_shape = content_image.shape
style_image = resize(style_image, target_shape)